In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
df = pd.read_csv(r"D:\jupyter notebook\nlp\codepth\datasets_72366_159129_BuzzFeed_fake_news_content.csv")

In [4]:
dff = pd.read_csv(r"D:\jupyter notebook\nlp\codepth\datasets_72366_159129_BuzzFeed_real_news_content.csv")

In [5]:
df.head()

,id,title,text,url,top_img,authors,source,publish_date,movies,images,canonical_link,meta_data
0,Fake_1-Webpage,Proof The Mainstream Media Is Manipulating The...,I woke up this morning to find a variation of ...,http://www.addictinginfo.org/2016/09/19/proof-...,http://addictinginfo.addictinginfoent.netdna-c...,Wendy Gittleson,http://www.addictinginfo.org,{'$date': 1474243200000},NaN,"http://i.imgur.com/JeqZLhj.png,http://addictin...",http://addictinginfo.com/2016/09/19/proof-the-...,"{""publisher"": ""Addicting Info | The Knowledge ..."
1,Fake_10-Webpage,Charity: Clinton Foundation Distributed “Water...,Former President Bill Clinton and his Clinton ...,http://eaglerising.com/36899/charity-clinton-f...,http://eaglerising.com/wp-content/uploads/2016...,View All Posts,http://eaglerising.com,{'$date': 1474416521000},NaN,http://constitution.com/wp-content/uploads/201...,http://eaglerising.com/36899/charity-clinton-f...,"{""description"": ""The possibility that CHAI dis..."
2,Fake_11-Webpage,A Hillary Clinton Administration May be Entire...,After collapsing just before trying to step in...,http://eaglerising.com/36880/a-hillary-clinton...,http://eaglerising.com/wp-content/uploads/2016...,"View All Posts,Tony Elliott",http://eaglerising.com,{'$date': 1474416638000},NaN,http://constitution.com/wp-content/uploads/201...,http://eaglerising.com/36880/a-hillary-clinton...,"{""description"": ""Hillary Clinton may be the fi..."
3,Fake_12-Webpage,Trump’s Latest Campaign Promise May Be His Mos...,"Donald Trump is, well, deplorable. He’s sugges...",http://www.addictinginfo.org/2016/09/19/trumps...,http://addictinginfo.addictinginfoent.netdna-c...,John Prager,http://www.addictinginfo.org,{'$date': 1474243200000},NaN,"http://i.imgur.com/JeqZLhj.png,http://2.gravat...",http://addictinginfo.com/2016/09/19/trumps-lat...,"{""publisher"": ""Addicting Info | The Knowledge ..."
4,Fake_13-Webpage,Website is Down For Maintenance,Website is Down For Maintenance,http://www.proudcons.com/clinton-foundation-ca...,NaN,NaN,http://www.proudcons.com,NaN,NaN,NaN,NaN,"{""og"": {""url"": ""http://www.proudcons.com"", ""ty..."


In [6]:
df['label'] = 'fake'

In [7]:
df = df[['title', 'text', 'url', 'authors', 'source', 'meta_data', 'label']]

In [8]:
df.head()

,title,text,url,authors,source,meta_data,label
0,Proof The Mainstream Media Is Manipulating The...,I woke up this morning to find a variation of ...,http://www.addictinginfo.org/2016/09/19/proof-...,Wendy Gittleson,http://www.addictinginfo.org,"{""publisher"": ""Addicting Info | The Knowledge ...",fake
1,Charity: Clinton Foundation Distributed “Water...,Former President Bill Clinton and his Clinton ...,http://eaglerising.com/36899/charity-clinton-f...,View All Posts,http://eaglerising.com,"{""description"": ""The possibility that CHAI dis...",fake
2,A Hillary Clinton Administration May be Entire...,After collapsing just before trying to step in...,http://eaglerising.com/36880/a-hillary-clinton...,"View All Posts,Tony Elliott",http://eaglerising.com,"{""description"": ""Hillary Clinton may be the fi...",fake
3,Trump’s Latest Campaign Promise May Be His Mos...,"Donald Trump is, well, deplorable. He’s sugges...",http://www.addictinginfo.org/2016/09/19/trumps...,John Prager,http://www.addictinginfo.org,"{""publisher"": ""Addicting Info | The Knowledge ...",fake
4,Website is Down For Maintenance,Website is Down For Maintenance,http://www.proudcons.com/clinton-foundation-ca...,NaN,http://www.proudcons.com,"{""og"": {""url"": ""http://www.proudcons.com"", ""ty...",fake


In [9]:
dff.head()

,id,title,text,url,top_img,authors,source,publish_date,movies,images,canonical_link,meta_data
0,Real_1-Webpage,Another Terrorist Attack in NYC…Why Are we STI...,"On Saturday, September 17 at 8:30 pm EST, an e...",http://eaglerising.com/36942/another-terrorist...,http://eaglerising.com/wp-content/uploads/2016...,"View All Posts,Leonora Cravotta",http://eaglerising.com,{'$date': 1474528230000},NaN,http://constitution.com/wp-content/uploads/201...,http://eaglerising.com/36942/another-terrorist...,"{""description"": ""\u201cWe believe at this poin..."
1,Real_10-Webpage,"Donald Trump: Drugs a 'Very, Very Big Factor' ...",Less than a day after protests over the police...,http://abcn.ws/2d4lNn9,http://a.abcnews.com/images/Politics/AP_donald...,"More Candace,Adam Kelsey,Abc News,More Adam",http://abcn.ws,NaN,NaN,http://www.googleadservices.com/pagead/convers...,http://abcnews.go.com/Politics/donald-trump-dr...,"{""fb_title"": ""Trump: Drugs a 'Very, Very Big F..."
2,Real_11-Webpage,"Obama To UN: ‘Giving Up Liberty, Enhances Secu...","Obama To UN: ‘Giving Up Liberty, Enhances Secu...",http://rightwingnews.com/barack-obama/obama-un...,http://rightwingnews.com/wp-content/uploads/20...,Cassy Fiano,http://rightwingnews.com,{'$date': 1474476044000},https://www.youtube.com/embed/ji6pl5Vwrvk,http://rightwingnews.com/wp-content/uploads/20...,http://rightwingnews.com/barack-obama/obama-un...,"{""googlebot"": ""noimageindex"", ""og"": {""site_nam..."
3,Real_12-Webpage,Trump vs. Clinton: A Fundamental Clash over Ho...,Getty Images Wealth Of Nations Trump vs. Clint...,http://politi.co/2de2qs0,http://static.politico.com/e9/11/6144cdc24e319...,"Jack Shafer,Erick Trickey,Zachary Karabell",http://politi.co,{'$date': 1474974420000},NaN,https://static.politico.com/dims4/default/8a1c...,http://www.politico.com/magazine/story/2016/09...,"{""description"": ""He sees it as zero-sum. She b..."
4,Real_13-Webpage,"President Obama Vetoes 9/11 Victims Bill, Sett...",President Obama today vetoed a bill that would...,http://abcn.ws/2dh2NFs,http://a.abcnews.com/images/US/AP_Obama_BM_201...,"John Parkinson,More John,Abc News,More Alexander",http://abcn.ws,NaN,NaN,http://www.googleadservices.com/pagead/convers...,http://abcnews.go.com/Politics/president-obama...,"{""fb_title"": ""President Obama Vetoes 9/11 Vict..."


In [10]:
dff['label'] = 'real'

In [11]:
dff = dff[['title', 'text', 'url', 'authors', 'source', 'meta_data', 'label']]

In [12]:
df.shape

(91, 7)

In [13]:
dff.shape

(91, 7)

In [14]:
df = pd.concat([df, dff])

In [15]:
df.shape

(182, 7)

In [16]:
df = df.sample(frac = 1)

In [17]:
df.head()

,title,text,url,authors,source,meta_data,label
17,"Post-debate, Trump team hits moderator Holt fo...","""He didn't ask her about a lot of things she s...",http://politi.co/2dojlvi,"Jack Shafer,Louis Nelson",http://politi.co,"{""description"": ""&quot;He didn't ask her about...",real
13,Anthony Weiner Under Federal Investigation for...,The U.S. Attorney’s Office in Manhattan is inv...,http://abcn.ws/2cVpv23,"More Josh,Abc News,Josh Margolin",http://abcn.ws,"{""fb_title"": ""Anthony Weiner Under Federal Inv...",real
54,Kaine visits site of Orlando mass shooting,Tim Kaine visits the Pulse night club in Orlan...,http://politi.co/2d1z7fz,"Jack Shafer,Burgess Everett",http://politi.co,"{""description"": ""Kaine's running mate, Hillary...",real
82,Joe Arpaio: Trump didn’t call Obama’s birth ce...,"“He didn’t say the document was legitimate,"" J...",http://politi.co/2cLospd,"Nick Gass,Madeline Conway,Jack Shafer",http://politi.co,"{""description"": ""\u201cIf he says the presiden...",real
8,Is it the Beginning of the End for Hillary Cli...,I can’t wait for “Weekend at Chelsea’s” to hit...,http://eaglerising.com/36912/is-it-the-beginni...,"View All Posts,Matt Barber",http://eaglerising.com,"{""description"": ""Am I saying there\u2019s no c...",fake


In [18]:
df.columns

Index(['title', 'text', 'url', 'authors', 'source', 'meta_data', 'label'], dtype='object')

In [19]:
from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
import re
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [20]:
df.columns

Index(['title', 'text', 'url', 'authors', 'source', 'meta_data', 'label'], dtype='object')

In [21]:
df['title'] = df['title'].apply(tweet_cleaner)
df['text'] = df['text'].apply(tweet_cleaner)
df['meta_data'] = df['meta_data'].apply(tweet_cleaner)

In [22]:
def all_X(row):
    try:
        return row['title']+" "+row["text"]+" "+row["meta_data"]
    except: print("error", row)
        #giving out the error fields.
        
        
df["all_X"]= df.apply(all_X,axis=1)

In [23]:
df = df[['all_X', 'label']]

In [24]:
df.columns

Index(['all_X', 'label'], dtype='object')

In [25]:
#Assign text to X variable and labels to y

X = df.all_X
y = df.label

In [26]:
#Intialize Count Vectorizer
count_vec = CountVectorizer(lowercase=True, stop_words="english", ngram_range=(1,2),
                           min_df = 3)
#Fit Count Vectorizer
dtm_cv = count_vec.fit_transform(X)
#Convert it to a pandas data frame
df_cv = pd.DataFrame(dtm_cv.toarray(), columns=count_vec.get_feature_names())

In [27]:
df_cv.head()

,abandon,abandoned,abc,abc abcnews,abc news,abcnews,abcnews abcnews,abcnews card,abcnews com,abcnews iphone,...,young,young adults,younger,zde,zde rd,zero,zuthf,zuthf dco,zyx,zyx bqy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,13,4,9,8,4,1,2,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
df_cv.shape

(182, 4191)

In [29]:
x_train, x_test, y_train, y_test = train_test_split(df_cv,y,test_size=0.1)

In [32]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
#Initialize model
model = MultinomialNB()
#Fit model with df_cv and y
model.fit(df_cv, y)
#score the model
y_pred = model.predict(x_test)
accuracy = cross_val_score(estimator = model, X = x_train, y = y_train, cv=10)
print("Model accuracy : {:0.2f}%".format(accuracy_score(y_pred,y_test)*100))
print("cross validation : {:0.2f}%".format(accuracy.mean()*100))

Model accuracy : 89.47%
cross validation : 84.26%
